
# Creating entangled states with optimized circuits for particular backend


## Introduction

Explanation

## Idea

...

## Solution

In [7]:
## Importing the necesary libraries

In [8]:
from qiskit import QuantumCircuit
from qiskit.providers.fake_provider import FakePrague, FakeSherbrooke, FakeAuckland , FakeQuitoV2
from qiskit import transpile
from qiskit.tools.visualization import plot_histogram
from qiskit.quantum_info import entropy, Statevector
from rustworkx.visualization import graphviz_draw
import rustworkx as rw
from rustworkx import minimum_spanning_edges, PyGraph, dijkstra_shortest_path_lengths
import matplotlib.pyplot as plt
import math
from random import randrange
from IPython import get_ipython

## Naive Approach

We will take the backend with a semnificative number of qubits, in order to have a statistically significant result.

In [9]:
my_variables = set(dir())


# Get a fake backend from the fake provider
prague = FakePrague()
sherbrooke = FakeSherbrooke()
auckland = FakeAuckland()
quitov2 = FakeQuitoV2()


# Create a simple circuit
num_qubits_sherbrooke = sherbrooke.num_qubits
circuit = QuantumCircuit(num_qubits_sherbrooke)
circuit.h(0)
for i in range(num_qubits_sherbrooke-1):
    circuit.cx(i,i+1)

# this circuit is way too large to be shown in a figure
circuit.measure_all()
# circuit.draw()

# Get the final state as a Statevector object
final_state = Statevector.from_instruction(circuit)

# Calculate the von-Neumann entropy of the final state
ent = entropy(final_state)

print(f'Entropy: {ent}')


naive_depth_sherbrooke = circuit.depth()
print(naive_depth_sherbrooke)




ValueError: Maximum allowed dimension exceeded

# Solution

In [ ]:
def get_optimized_state_random(backend):
    findme = 'num_qubits'
    if findme in dir(backend):
        num_qubits = backend.num_qubits
        connections = backend.coupling_map
    else:
        num_qubits = backend.configuration().n_qubits
        connections = backend.configuration().coupling_map
    
    # Transpile the ideal circuit to a circuit that can be directly executed by the backend
    qc = QuantumCircuit(num_qubits)
    transpiled_circuit = transpile(qc, backend)
    transpiled_circuit.draw('mpl')

    # Run the transpiled circuit using the simulated fake backend
    job = backend.run(transpiled_circuit)
    # counts = job.result().get_counts()
    # plot_histogram(counts)
    print("Coupling map: ", connections)

    targeted = {}
    for i in range(num_qubits):
        targeted.update({i: 'n'})
    print(targeted)
    start = randrange(num_qubits)
    print("Start qubit: ", start)

    qc.h(start)
    # print(qc.draw())
    # mark this qubit as 'targeted'
    targeted[start] = 't'

    list_ctrl = []
    list_ctrl.append(start)
    # print("Initial list of control qubits: ", list_ctrl)
    # print("First element of the list: ", list_ctrl[0])

    def non_entangled_exist(target_dict):
        ret = False
        for i in range(len(target_dict)):
            if target_dict[i] == 'n':
                ret = True

        return ret

    iteration = 0
    while non_entangled_exist(targeted):
        iteration += 1
        print("Iteration: ", iteration)
        print("=============")
        print("Targeted: ", targeted)
        # helper list_ctrl to be used for swap at the end of the loop
        swp_list_ctrl = []
        # for each node in list_ctrl find connected nodes which haven't been entangled (targeted)
        for i in list_ctrl:
            print(i)
            # find connected nodes in connections not targeted
            for j in connections:
                # apply cx on it
                # mark it as targeted
                # add it to swp_list_ctrl
                if j[0] == i:
                    if targeted[j[1]] == 'n':
                        qc.cx(i, j[1])
                        targeted[j[1]] = 't'
                        swp_list_ctrl.append(j[1])
                if j[1] == i:
                    if targeted[j[0]] == 'n':
                        qc.cx(i, j[0])
                        targeted[j[0]] = 't'
                        swp_list_ctrl.append(j[0])
        # clear list_ctrl
        list_ctrl = []
        # replace list_ctrl with swp_list_ctrl
        for i in swp_list_ctrl:
            list_ctrl.append(i)
        qc.barrier()
   
    # Get the final state as a Statevector object
    final_state = Statevector.from_instruction(qc)

    # Calculate the von-Neumann entropy of the final state
    ent = entropy(final_state)

    print(f'Entropy: {ent}')    
    # print(qc.draw())
    # print("The circuit depth: ", qc.depth())

    return [qc.depth(), num_qubits]

# Testing the solution on the Sherbrooke backend with a random initial state

print("The circuit depth: ", get_optimized_state_random(sherbrooke)[0])

Coupling map:  [[55, 49], [73, 66], [92, 83], [99, 100], [117, 116], [22, 15], [26, 16], [31, 30], [33, 39], [35, 47], [43, 34], [53, 60], [64, 54], [71, 77], [81, 72], [93, 87], [98, 91], [111, 104], [114, 109], [5, 4], [16, 8], [17, 12], [21, 20], [34, 24], [35, 28], [41, 40], [50, 51], [54, 45], [61, 62], [69, 68], [71, 58], [73, 85], [80, 79], [89, 88], [92, 102], [107, 106], [109, 96], [110, 118], [122, 111], [126, 125], [4, 3], [8, 9], [12, 13], [14, 0], [20, 33], [23, 24], [28, 27], [31, 32], [42, 41], [45, 46], [49, 50], [52, 37], [59, 58], [66, 65], [69, 70], [72, 62], [78, 79], [83, 84], [87, 88], [90, 75], [97, 96], [104, 103], [107, 108], [110, 100], [120, 121], [125, 124], [3, 2], [7, 6], [10, 11], [14, 18], [21, 22], [26, 25], [29, 30], [40, 39], [43, 44], [47, 48], [56, 52], [59, 60], [63, 64], [68, 67], [78, 77], [81, 82], [86, 85], [90, 94], [97, 98], [101, 102], [106, 105], [114, 115], [119, 118], [123, 122], [1, 2], [6, 5], [10, 9], [17, 30], [18, 19], [23, 22], [26,

## Selecting a the qubit corresponding to the root node in the spanning tree

In [ ]:
def get_optimized_state_root(backend):
    findme = 'num_qubits'
    if findme in dir(backend):
        num_qubits = backend.num_qubits
        connections = backend.coupling_map
    else:
        num_qubits = backend.configuration().n_qubits
        connections = backend.configuration().coupling_map
    
    # Transpile the ideal circuit to a circuit that can be directly executed by the backend
    qc = QuantumCircuit(num_qubits)
    transpiled_circuit = transpile(qc, backend)
    transpiled_circuit.draw('mpl')

    # Run the transpiled circuit using the simulated fake backend
    job = backend.run(transpiled_circuit)
    
    # counts = job.result().get_counts()
    # plot_histogram(counts)

    print("Coupling map: ", connections)


    def edge_cost_fn(weight):
        # return math.exp(weight)
        return weight
    
    graph = rw.PyGraph()
    edges = connections.get_edges()
    weighted_edges = [(source, target, randrange(1,100)/1000) for source, target in edges]
    graph.extend_from_weighted_edge_list(weighted_edges)
    
    # paths = rw.graph_all_pairs_dijkstra_shortest_paths(graph, edge_cost_fn)

    min_max_length = float('inf')
    centroid = None
    for node in graph.node_indices():
        lengths = dijkstra_shortest_path_lengths(graph, node, edge_cost_fn)
        max_length = max(lengths.values())
        if max_length < min_max_length:
            min_max_length = max_length
            centroid = node

    print(f"Centroid: {centroid}")

    targeted = {}
    for i in range(num_qubits):
        targeted.update({i: 'n'})
    print(targeted)
    start = randrange(num_qubits)
    print("Start qubit: ", start)

    qc.h(start)
    # print(qc.draw())
    # mark this qubit as 'targeted'
    targeted[start] = 't'

    list_ctrl = []
    list_ctrl.append(start)
    # print("Initial list of control qubits: ", list_ctrl)
    # print("First element of the list: ", list_ctrl[0])

    def non_entangled_exist(target_dict):
        ret = False
        for i in range(len(target_dict)):
            if target_dict[i] == 'n':
                ret = True

        return ret

    iteration = 0
    while non_entangled_exist(targeted):
        iteration += 1
        print("Iteration: ", iteration)
        print("=============")
        print("Targeted: ", targeted)
        # helper list_ctrl to be used for swap at the end of the loop
        swp_list_ctrl = []
        # for each node in list_ctrl find connected nodes which haven't been entangled (targeted)
        for i in list_ctrl:
            print(i)
            # find connected nodes in connections not targeted
            for j in connections:
                # apply cx on it
                # mark it as targeted
                # add it to swp_list_ctrl
                if j[0] == i:
                    if targeted[j[1]] == 'n':
                        qc.cx(i, j[1])
                        targeted[j[1]] = 't'
                        swp_list_ctrl.append(j[1])
                if j[1] == i:
                    if targeted[j[0]] == 'n':
                        qc.cx(i, j[0])
                        targeted[j[0]] = 't'
                        swp_list_ctrl.append(j[0])
        # clear list_ctrl
        list_ctrl = []
        # replace list_ctrl with swp_list_ctrl
        for i in swp_list_ctrl:
            list_ctrl.append(i)
        qc.barrier()
        
    # Get the final state as a Statevector object
    final_state = Statevector.from_instruction(qc)

    # Calculate the von-Neumann entropy of the final state
    ent = entropy(final_state)

    print(f'Entropy: {ent}')

    return [qc.depth(), num_qubits]


    # graphviz_draw(graph,method='neato')
    graphviz_draw(rw.minimum_spanning_tree(graph,weight_fn=edge_cost_fn),method = 'dot')

# Testing the solution on the Sherbrooke fake backend with the starting qubit as the root of the spanning tree
print("The circuit depth: ", get_optimized_state_root(sherbrooke)[0])


Coupling map:  [[55, 49], [73, 66], [92, 83], [99, 100], [117, 116], [22, 15], [26, 16], [31, 30], [33, 39], [35, 47], [43, 34], [53, 60], [64, 54], [71, 77], [81, 72], [93, 87], [98, 91], [111, 104], [114, 109], [5, 4], [16, 8], [17, 12], [21, 20], [34, 24], [35, 28], [41, 40], [50, 51], [54, 45], [61, 62], [69, 68], [71, 58], [73, 85], [80, 79], [89, 88], [92, 102], [107, 106], [109, 96], [110, 118], [122, 111], [126, 125], [4, 3], [8, 9], [12, 13], [14, 0], [20, 33], [23, 24], [28, 27], [31, 32], [42, 41], [45, 46], [49, 50], [52, 37], [59, 58], [66, 65], [69, 70], [72, 62], [78, 79], [83, 84], [87, 88], [90, 75], [97, 96], [104, 103], [107, 108], [110, 100], [120, 121], [125, 124], [3, 2], [7, 6], [10, 11], [14, 18], [21, 22], [26, 25], [29, 30], [40, 39], [43, 44], [47, 48], [56, 52], [59, 60], [63, 64], [68, 67], [78, 77], [81, 82], [86, 85], [90, 94], [97, 98], [101, 102], [106, 105], [114, 115], [119, 118], [123, 122], [1, 2], [6, 5], [10, 9], [17, 30], [18, 19], [23, 22], [26,

# Results

In [ ]:
results = []

backends = [ prague, quitov2, auckland, sherbrooke ]

for backend in backends:
    result = get_optimized_state_root(backend)
    results.append(1-result[0]/result[1])

print(results)


Coupling map:  [[2, 3], [3, 2], [4, 7], [7, 4], [8, 11], [11, 8], [12, 13], [13, 12], [16, 19], [19, 16], [18, 21], [21, 18], [24, 25], [25, 24], [27, 28], [28, 27], [1, 4], [4, 1], [3, 30], [30, 3], [8, 9], [9, 8], [10, 12], [12, 10], [14, 16], [16, 14], [17, 18], [18, 17], [22, 25], [25, 22], [23, 27], [27, 23], [1, 2], [2, 1], [5, 8], [8, 5], [7, 10], [10, 7], [13, 14], [14, 13], [15, 18], [18, 15], [19, 22], [22, 19], [23, 24], [24, 23], [30, 31], [31, 30], [0, 1], [1, 0], [3, 5], [5, 3], [6, 7], [7, 6], [11, 14], [14, 11], [12, 15], [15, 12], [19, 20], [20, 19], [21, 23], [23, 21], [25, 26], [26, 25], [28, 29], [29, 28], [31, 32], [32, 31]]
Centroid: 12
{0: 'n', 1: 'n', 2: 'n', 3: 'n', 4: 'n', 5: 'n', 6: 'n', 7: 'n', 8: 'n', 9: 'n', 10: 'n', 11: 'n', 12: 'n', 13: 'n', 14: 'n', 15: 'n', 16: 'n', 17: 'n', 18: 'n', 19: 'n', 20: 'n', 21: 'n', 22: 'n', 23: 'n', 24: 'n', 25: 'n', 26: 'n', 27: 'n', 28: 'n', 29: 'n', 30: 'n', 31: 'n', 32: 'n'}
Start qubit:  23
Iteration:  1
Targeted:  {0: